<a href="https://colab.research.google.com/github/oliviafelica/pulu-pulu/blob/main/ga-phre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
#import library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import datetime
from plotly.subplots import make_subplots

from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from statsmodels.tsa.vector_ar.var_model import VAR

from warnings import filterwarnings
filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import make_scorer
from keras.wrappers.scikit_learn import KerasRegressor

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.statespace.exponential_smoothing import ExponentialSmoothing
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
import itertools
import warnings
from joblib import dump

In [45]:
#memanggil semua package
import os
from tensorflow.python.keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU
from sklearn.impute import SimpleImputer # used for handling missing data
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from keras import optimizers
from tensorflow.keras.optimizers import Adam

#DATA

In [136]:
#data
data = pd.read_excel('/content/Data - Estate Area 3A-3B edit.xlsx',
sheet_name = '3A-PHRE')
data

,Area 3A,estate,tahun,bulan,date,tonnase,luas,yield,fit_yield,CH,HH,WB,WA1,RUT,kelas_RUT,soil
0,Area 3A,PHRE,2010,JAN,2010-01-01,2808.98,4429.84,0.634104,0.634104,440.85,20.0,520.85,200.0,5.792646,muda,Mineral
1,Area 3A,PHRE,2010,FEB,2010-02-01,2748.76,4429.84,0.620510,0.627307,259.30,17.0,339.30,200.0,5.792646,muda,Mineral
2,Area 3A,PHRE,2010,MAR,2010-03-01,3168.74,4429.84,0.715317,0.656644,394.80,22.0,474.80,200.0,5.792646,muda,Mineral
3,Area 3A,PHRE,2010,APR,2010-04-01,3281.61,4429.84,0.740797,0.677682,294.40,17.0,374.40,200.0,5.792646,muda,Mineral
4,Area 3A,PHRE,2010,MAY,2010-05-01,2979.29,4429.84,0.672550,0.676656,271.90,17.0,351.90,200.0,5.792646,muda,Mineral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,Area 3A,PHRE,2022,AUG,2022-08-01,7451.83,2770.91,2.689308,1.915866,473.68,17.0,553.68,200.0,17.863373,tua,Mineral
152,Area 3A,PHRE,2022,SEP,2022-09-01,6193.74,2770.91,2.235273,1.951355,439.84,22.0,519.44,200.0,17.863373,tua,Mineral
153,Area 3A,PHRE,2022,OCT,2022-10-01,6149.06,2770.91,2.219148,1.978135,284.70,25.0,364.70,200.0,17.863373,tua,Mineral
154,Area 3A,PHRE,2022,NOV,2022-11-01,6006.97,2770.91,2.167869,1.995383,345.75,22.0,425.75,200.0,17.863373,tua,Mineral


In [137]:
data['tahun'] = [str(number) for number in data['tahun']]

##data.info()
bulan = data['bulan']
tahun = data['tahun']

# Create a list of mm/yyyy strings
date_strings = [f"{month}-{year}" for year, month in zip(tahun, bulan)]

# Define a function to convert the mm/yyyy strings to datetime objects
def convert_to_datetime(date_str):
    return pd.to_datetime(date_str, format="%b-%Y")

# Convert the list of mm/yyyy strings to a list of datetime objects
date_objects = [convert_to_datetime(date_str) for date_str in date_strings]

# Add the converted datetime objects to the DataFrame
data['date'] = date_objects

# Format the 'date_column' to "mm/yyyy" as strings
data['date'] = data['date'].dt.strftime("%m/%Y")

data.set_index('date', inplace = True)
null = data.isna().any(axis=1)
rows_with_nan = data[null]
# # split data
# # Specify the lag structure for 'Drought' and 'Rainfall'
# max_lag = 24  # Adjust this based on domain knowledge or model selection criteria
# max_lag_yield = 11
# # create lagged variables for 'WA1' and 'CH'
# for lag in range(1, max_lag + 1):
#     data[f'WA1_{lag}'] = data['WA1'].shift(lag)
#     data[f'CH_{lag}'] = data['CH'].shift(lag)
#     data[f'HH_{lag}'] = data['HH'].shift(lag)
#     data[f'RUT_{lag}'] = data['HH'].shift(lag)

# for lag in range(1, max_lag_yield + 1):
#     data[f'yield_{lag}'] = data['yield'].shift(lag)

data22 = data.iloc[-12:]
data = data.iloc[:-12]
# # Menghapus baris dengan nilai NaN setelah membuat variabel lag
data = data.dropna()

# data.info()

In [138]:
data22.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 01/2022 to 12/2022
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Area 3A    12 non-null     object 
 1   estate     12 non-null     object 
 2   tahun      12 non-null     object 
 3   bulan      12 non-null     object 
 4   tonnase    12 non-null     float64
 5   luas       12 non-null     float64
 6   yield      12 non-null     float64
 7   fit_yield  12 non-null     float64
 8   CH         12 non-null     float64
 9   HH         12 non-null     float64
 10  WB         12 non-null     float64
 11  WA1        12 non-null     float64
 12  RUT        12 non-null     float64
 13  kelas_RUT  12 non-null     object 
 14  soil       12 non-null     object 
dtypes: float64(9), object(6)
memory usage: 1.5+ KB


In [139]:
data = data.drop(columns = ['Area 3A', 'estate', 'bulan', 'tahun', 'bulan', 'tonnase', 'fit_yield', 'kelas_RUT', 'soil'])

#XGBOOST

In [140]:
import pandas as pd
import numpy as np
import random
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Fungsi untuk menghitung mean absolute percentage error (MAPE)
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Membaca data dari file Excel
# def read_data(file_path):
#     data = pd.read_excel(file_path, sheet_name= '3A-PHRE')
#     data['Date'] = pd.to_datetime(data['Date'])
#     data.set_index('Date', inplace=True)
#     data = data.drop(columns=['RUT_Category'])
#     return data

# Preprocessing data
def preprocess_data(data):
    # Menambahkan lag features ke dalam data
    for i in range(1, 25):
        data[f'WA1_lag{i}'] = data['WA1'].shift(i)
        data[f'CH_lag{i}'] = data['CH'].shift(i)
        data[f'HH_lag{i}'] = data['HH'].shift(i)

    # Menghapus baris yang memiliki nilai NaN
    data = data.dropna()
    return data

# Algoritma Genetika
def genetic_algorithm(X_train, y_train, X_test, y_test, population_size, num_generations, num_parents, mutation_rate, random_seed=100):
    # Set random seed untuk hasil yang konsisten
    random.seed(random_seed)
    np.random.seed(random_seed)

    # Inisialisasi populasi awal dengan chromosome yang acak
    def initialize_population(n, num_features):
        population = []
        for _ in range(n):
            chromosome = [random.randint(0, 1) for _ in range(num_features)]
            population.append(chromosome)
        return population

    # Fungsi untuk menghitung fitness dari chromosome
    def calculate_fitness(chromosome):
        selected_features = [i for i in range(len(chromosome)) if chromosome[i] == 1]
        if len(selected_features) == 0:
            return -1  # Nilai fitness default jika tidak ada fitur yang dipilih
        else:
            # Membangun model XGBoost dengan fitur yang dipilih
            model = xgb.XGBRegressor(n_estimators=100, random_state=random_seed)
            model.fit(X_train.iloc[:, selected_features], y_train)
            y_pred = model.predict(X_test.iloc[:, selected_features])
            mape = mean_absolute_percentage_error(y_test, y_pred)
            return 1 / (mape + 1e-6)  # Menggunakan 1/MAPE untuk maksimisasi fitness

    # Seleksi orangtua berdasarkan fitness
    def select_parents(population, num_parents):
        fitness_scores = [calculate_fitness(chromosome) for chromosome in population]
        selected_parents_indices = np.argsort(fitness_scores)[-num_parents:]
        selected_parents = [population[i] for i in selected_parents_indices]
        return selected_parents

    # Mutasi chromosome
    def mutate(chromosome, mutation_rate):
        for i in range(len(chromosome)):
            if random.random() < mutation_rate:
                chromosome[i] = 1 - chromosome[i]
        return chromosome

    num_features = X_train.shape[1]
    population = initialize_population(population_size, num_features)

    # Melakukan iterasi untuk generasi-genarasi selanjutnya
    for generation in range(num_generations):
        parents = select_parents(population, num_parents)
        children = []
        while len(children) < population_size:
            parent1, parent2 = random.choices(parents, k=2)
            crossover_point = random.randint(1, num_features - 1)
            child = parent1[:crossover_point] + parent2[crossover_point:]
            child = mutate(child, mutation_rate)
            children.append(child)
        population = children

    # Memilih chromosome terbaik sebagai fitur terpilih
    best_chromosome = max(population, key=calculate_fitness)
    selected_features = [i for i in range(len(best_chromosome)) if best_chromosome[i] == 1]

    return selected_features

# Main program
if __name__ == "__main__":
    # file_path = '/content/Data - Estate Area 3A-3B edit.xlsx'
    # data = read_data(file_path)
    data = preprocess_data(data)

    # Menentukan variabel dependen dan independen untuk pelatihan
    train_data = data.iloc[:108]
    test_data = data.iloc[108:]

    y_train = train_data['yield']
    X_train = train_data.drop(columns=['yield'])

    # Menentukan variabel dependen dan independen untuk pengujian
    y_test = test_data['yield']
    X_test = test_data.drop(columns=['yield'])

    # Penskalaan data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

    # Algoritma Genetika
    selected_features = genetic_algorithm(X_train, y_train, X_test, y_test, population_size=50, num_generations=30, num_parents=20, mutation_rate=0.01, random_seed=100)

    # Melatih model XGBoost dengan fitur terpilih
    model = xgb.XGBRegressor(n_estimators=100, random_state=100)
    model.fit(X_train.iloc[:, selected_features], y_train)

    # Evaluasi model pada testing set
    y_pred = model.predict(X_test.iloc[:, selected_features])
    mape = mean_absolute_percentage_error(y_test, y_pred)

    print("Fitur Terpilih:", selected_features)
    print("Mean Absolute Percentage Error (MAPE):", mape)


Fitur Terpilih: [0, 1, 2, 4, 5, 7, 12, 13, 14, 15, 19, 20, 23, 25, 30, 31, 36, 37, 38, 39, 40, 42, 43, 44, 45, 50, 53, 61, 75]
Mean Absolute Percentage Error (MAPE): 7.551493491055976


In [84]:
import joblib
from joblib import load, dump
dump(scaler, 'scaler_x_phre.joblib')

['scaler_x_phre.joblib']

In [56]:
import joblib
# save
joblib.dump(model, "xgboost_phre.joblib")

# load
#loaded_rf = joblib.load("my_random_forest.joblib")

['xgboost_phre.joblib']

In [82]:
# Assuming 'selected_features' contains the indices of selected features
selected_feature_names = [X_train.columns[i] for i in selected_features]

# Now, 'selected_feature_names' contains the names of the selected features
selected_feature_names

['luas',
 'CH',
 'HH',
 'WA1',
 'RUT',
 'CH_lag1',
 'WA1_lag3',
 'CH_lag3',
 'HH_lag3',
 'WA1_lag4',
 'CH_lag5',
 'HH_lag5',
 'HH_lag6',
 'CH_lag7',
 'WA1_lag9',
 'CH_lag9',
 'WA1_lag11',
 'CH_lag11',
 'HH_lag11',
 'WA1_lag12',
 'CH_lag12',
 'WA1_lag13',
 'CH_lag13',
 'HH_lag13',
 'WA1_lag14',
 'HH_lag15',
 'HH_lag16',
 'CH_lag19',
 'WA1_lag24']

#LINREG

In [94]:
import pandas as pd
import numpy as np
import random
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Fungsi untuk menghitung mean absolute percentage error (MAPE)
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# # Membaca data dari file Excel
# def read_data(file_path):
#     data = pd.read_excel(file_path, parse_dates=['Date'])
#     data['Date'] = pd.to_datetime(data['Date'])
#     data.set_index('Date', inplace=True)
#     data = data.drop(columns=['RUT_Category'])
#     return data

# Preprocessing data
def preprocess_data(data):
    # Menambahkan lag features ke dalam data
    for i in range(1, 25):
        data[f'WA1_lag{i}'] = data['WA1'].shift(i)
        data[f'CH_lag{i}'] = data['CH'].shift(i)
        data[f'HH_lag{i}'] = data['HH'].shift(i)

    # Menghapus baris yang memiliki nilai NaN
    data = data.dropna()
    return data

# Algoritma Genetika
def genetic_algorithm(X_train, y_train, X_test, y_test, population_size, num_generations, num_parents, mutation_rate, random_seed=100):
    # Set random seed untuk hasil yang konsisten
    random.seed(random_seed)
    np.random.seed(random_seed)

    # Inisialisasi populasi awal dengan chromosome yang acak
    def initialize_population(n, num_features):
        population = []
        for _ in range(n):
            chromosome = [random.randint(0, 1) for _ in range(num_features)]
            population.append(chromosome)
        return population

    # Fungsi untuk menghitung fitness dari chromosome
    def calculate_fitness(chromosome):
        selected_features = [i for i in range(len(chromosome)) if chromosome[i] == 1]
        if len(selected_features) == 0:
            return -1  # Nilai fitness default jika tidak ada fitur yang dipilih
        else:
            # Membangun model Linear Regression dengan fitur yang dipilih
            model = LinearRegression()
            model.fit(X_train.iloc[:, selected_features], y_train)
            y_pred = model.predict(X_test.iloc[:, selected_features])
            mape = mean_absolute_percentage_error(y_test, y_pred)
            return 1 / (mape + 1e-6)  # Menggunakan 1/MAPE untuk maksimisasi fitness

    # Seleksi orangtua berdasarkan fitness
    def select_parents(population, num_parents):
        fitness_scores = [calculate_fitness(chromosome) for chromosome in population]
        selected_parents_indices = np.argsort(fitness_scores)[-num_parents:]
        selected_parents = [population[i] for i in selected_parents_indices]
        return selected_parents

    # Mutasi chromosome
    def mutate(chromosome, mutation_rate):
        for i in range(len(chromosome)):
            if random.random() < mutation_rate:
                chromosome[i] = 1 - chromosome[i]
        return chromosome

    num_features = X_train.shape[1]
    population = initialize_population(population_size, num_features)

    # Melakukan iterasi untuk generasi-genarasi selanjutnya
    for generation in range(num_generations):
        parents = select_parents(population, num_parents)
        children = []
        while len(children) < population_size:
            parent1, parent2 = random.choices(parents, k=2)
            crossover_point = random.randint(1, num_features - 1)
            child = parent1[:crossover_point] + parent2[crossover_point:]
            child = mutate(child, mutation_rate)
            children.append(child)
        population = children

    # Memilih chromosome terbaik sebagai fitur terpilih
    best_chromosome = max(population, key=calculate_fitness)
    selected_features = [i for i in range(len(best_chromosome)) if best_chromosome[i] == 1]

    return selected_features

# Main program
if __name__ == "__main__":
    # file_path = '/content/Data - Estate Area 3A-3B edit.xlsx'
    # data = read_data(file_path)
    data = preprocess_data(data)

    # Menentukan variabel dependen dan independen untuk pelatihan
    train_data = data.iloc[:108]
    test_data = data.iloc[108:]

    y_train = train_data['yield']
    X_train = train_data.drop(columns=['yield'])

    # Menentukan variabel dependen dan independen untuk pengujian
    y_test = test_data['yield']
    X_test = test_data.drop(columns=['yield'])

    # Penskalaan data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

    # Algoritma Genetika
    selected_features = genetic_algorithm(X_train, y_train, X_test, y_test, population_size=50, num_generations=30, num_parents=20, mutation_rate=0.01, random_seed=100)

    # Melatih model Linear Regression dengan fitur terpilih
    model_lin = LinearRegression()
    model_lin.fit(X_train.iloc[:, selected_features], y_train)

    # Evaluasi model pada testing set
    y_pred = model_lin.predict(X_test.iloc[:, selected_features])
    mape = mean_absolute_percentage_error(y_test, y_pred)

    print("Fitur Terpilih:", selected_features)
    print("Mean Absolute Percentage Error (MAPE):", mape)

Fitur Terpilih: [1, 2, 3, 4, 7, 11, 14, 16, 17, 19, 24, 29, 30, 31, 33, 36, 38, 40, 42, 45, 46, 47, 51, 55, 56, 58, 59, 64, 65, 69, 71, 72, 74, 75, 77]
Mean Absolute Percentage Error (MAPE): 10.336577416064612


# RANDOM FOREST

In [134]:
GradientBoostingRegressor,
import pandas as pd
import numpy as np
import random
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Fungsi untuk menghitung mean absolute percentage error (MAPE)
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# # Membaca data dari file Excel
# def read_data(file_path):
#     data = pd.read_excel(file_path, parse_dates=['Date'])
#     data['Date'] = pd.to_datetime(data['Date'])
#     data.set_index('Date', inplace=True)
#     data = data.drop(columns=['RUT_Category'])
#     return data

# Preprocessing data
def preprocess_data(data):
    # Menambahkan lag features ke dalam data
    for i in range(1, 25):
        data[f'WA1_lag{i}'] = data['WA1'].shift(i)
        data[f'CH_lag{i}'] = data['CH'].shift(i)
        data[f'HH_lag{i}'] = data['HH'].shift(i)

    # Menghapus baris yang memiliki nilai NaN
    data = data.dropna()
    return data

# Algoritma Genetika
def genetic_algorithm(X_train, y_train, X_test, y_test, population_size, num_generations, num_parents, mutation_rate, random_seed=100):
    # Set random seed untuk hasil yang konsisten
    random.seed(random_seed)
    np.random.seed(random_seed)

    # Inisialisasi populasi awal dengan chromosome yang acak
    def initialize_population(n, num_features):
        population = []
        for _ in range(n):
            chromosome = [random.randint(0, 1) for _ in range(num_features)]
            population.append(chromosome)
        return population

    # Fungsi untuk menghitung fitness dari chromosome
    def calculate_fitness(chromosome):
        selected_features = [i for i in range(len(chromosome)) if chromosome[i] == 1]
        if len(selected_features) == 0:
            return -1  # Nilai fitness default jika tidak ada fitur yang dipilih
        else:
            # Membangun model Linear Regression dengan fitur yang dipilih
            model = LinearRegression()
            model.fit(X_train.iloc[:, selected_features], y_train)
            y_pred = model.predict(X_test.iloc[:, selected_features])
            mape = mean_absolute_percentage_error(y_test, y_pred)
            return 1 / (mape + 1e-6)  # Menggunakan 1/MAPE untuk maksimisasi fitness

    # Seleksi orangtua berdasarkan fitness
    def select_parents(population, num_parents):
        fitness_scores = [calculate_fitness(chromosome) for chromosome in population]
        selected_parents_indices = np.argsort(fitness_scores)[-num_parents:]
        selected_parents = [population[i] for i in selected_parents_indices]
        return selected_parents

    # Mutasi chromosome
    def mutate(chromosome, mutation_rate):
        for i in range(len(chromosome)):
            if random.random() < mutation_rate:
                chromosome[i] = 1 - chromosome[i]
        return chromosome

    num_features = X_train.shape[1]
    population = initialize_population(population_size, num_features)

    # Melakukan iterasi untuk generasi-genarasi selanjutnya
    for generation in range(num_generations):
        parents = select_parents(population, num_parents)
        children = []
        while len(children) < population_size:
            parent1, parent2 = random.choices(parents, k=2)
            crossover_point = random.randint(1, num_features - 1)
            child = parent1[:crossover_point] + parent2[crossover_point:]
            child = mutate(child, mutation_rate)
            children.append(child)
        population = children

    # Memilih chromosome terbaik sebagai fitur terpilih
    best_chromosome = max(population, key=calculate_fitness)
    selected_features = [i for i in range(len(best_chromosome)) if best_chromosome[i] == 1]

    return selected_features

# Main program
if __name__ == "__main__":
    # file_path = '/content/Data - Estate Area 3A-3B edit.xlsx'
    # data = read_data(file_path)
    data = preprocess_data(data)

    # Menentukan variabel dependen dan independen untuk pelatihan
    train_data = data.iloc[:108]
    test_data = data.iloc[108:]

    y_train = train_data['yield']
    X_train = train_data.drop(columns=['yield'])

    # Menentukan variabel dependen dan independen untuk pengujian
    y_test = test_data['yield']
    X_test = test_data.drop(columns=['yield'])

    # Penskalaan data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

    # Algoritma Genetika
    selected_features = genetic_algorithm(X_train, y_train, X_test, y_test, population_size=50, num_generations=30, num_parents=20, mutation_rate=0.01, random_seed=100)

    # Melatih model Linear Regression dengan fitur terpilih
    model = DecisionTreeRegressor()
    model.fit(X_train.iloc[:, selected_features], y_train)

    # Evaluasi model pada testing set
    y_pred = model.predict(X_test.iloc[:, selected_features])
    mape = mean_absolute_percentage_error(y_test, y_pred)

    print("Fitur Terpilih:", selected_features)
    print("Mean Absolute Percentage Error (MAPE):", mape)

Fitur Terpilih: [1, 2, 3, 4, 7, 11, 14, 16, 17, 19, 24, 29, 30, 31, 33, 36, 38, 40, 42, 45, 46, 47, 51, 55, 56, 58, 59, 64, 65, 69, 71, 72, 74, 75, 77]
Mean Absolute Percentage Error (MAPE): 23.00403386261232


#2023 Forecast

In [81]:
# Specify the lag structure for 'Drought' and 'Rainfall'
max_lag = 24  # Adjust this based on domain knowledge or model selection criteria

for lag in range(1, max_lag + 1):
    data22[f'WA1_{lag}'] = data22['WA1'].shift(lag)
    data22[f'CH_{lag}'] = data22['CH'].shift(lag)
    data22[f'HH_{lag}'] = data22['HH'].shift(lag)

In [73]:
data22.

,Area 3A,estate,tahun,bulan,tonnase,luas,yield,fit_yield,CH,HH,WB,WA1,RUT,kelas_RUT,soil
date,,,,,,,,,,,,,,,
01/2022,Area 3A,PHRE,2022,JAN,2942.74,2770.91,1.062012,1.062012,189.20,14.0,269.20,200.0,17.863373,tua,Mineral
02/2022,Area 3A,PHRE,2022,FEB,3521.90,2770.91,1.271026,1.166519,177.50,16.0,257.50,200.0,17.863373,tua,Mineral
03/2022,Area 3A,PHRE,2022,MAR,4629.84,2770.91,1.670873,1.334637,210.10,19.0,290.10,200.0,17.863373,tua,Mineral
04/2022,Area 3A,PHRE,2022,APR,5783.27,2770.91,2.087137,1.522762,164.90,17.0,244.90,200.0,17.863373,tua,Mineral
05/2022,Area 3A,PHRE,2022,MAY,5565.39,2770.91,2.008506,1.619911,281.00,14.0,361.00,200.0,17.863373,tua,Mineral
06/2022,Area 3A,PHRE,2022,JUN,6627.03,2770.91,2.391644,1.748533,299.40,19.0,379.40,200.0,17.863373,tua,Mineral
07/2022,Area 3A,PHRE,2022,JUL,5947.53,2770.91,2.146418,1.805374,396.60,14.0,476.60,200.0,17.863373,tua,Mineral
08/2022,Area 3A,PHRE,2022,AUG,7451.83,2770.91,2.689308,1.915866,473.68,17.0,553.68,200.0,17.863373,tua,Mineral
09/2022,Area 3A,PHRE,2022,SEP,6193.74,2770.91,2.235273,1.951355,439.84,22.0,519.44,200.0,17.863373,tua,Mineral


In [76]:
#data
data = pd.read_excel('/content/X.xlsx')
data

,Date,Yield,Luasan,RUT,RUT_Category,WA,CH,HH
0,2023-01-01,NaN,2811.72,22.514597,tua,200.000000,243.775385,18.076923
1,2023-02-01,NaN,2811.72,22.514597,tua,200.000000,205.590000,15.384615
2,2023-03-01,NaN,2811.72,22.514597,tua,200.000000,346.403077,22.230769
3,2023-04-01,NaN,2811.72,22.514597,tua,200.000000,176.856154,14.538462
4,2023-05-01,NaN,2811.72,22.514597,tua,195.380000,138.523846,10.307692
5,2023-06-01,NaN,2811.72,22.514597,tua,200.000000,188.555385,11.230769
6,2023-07-01,NaN,2811.72,22.514597,tua,200.000000,250.217692,12.153846
7,2023-08-01,NaN,2811.72,22.514597,tua,135.155385,85.155385,6.846154
8,2023-09-01,NaN,2811.72,22.514597,tua,200.000000,165.100000,11.000000
9,2023-10-01,NaN,2811.72,22.514597,tua,200.000000,257.302000,24.000000
